In [1]:
import pandas as pd

import config

In [2]:
CFG = config.Config()

In [3]:
dataset: pd.DataFrame = (
    pd.read_parquet(CFG.final_data_files["user_content"])
    ["content"]
    .drop_duplicates()
    .pipe(lambda _se: _se[_se.str.len() >= 4])
    .sample(frac=1.)
)
dataset

id
662a389637d6395f42ca9e80    Ihr wart noch nicht in häufig in Frankfurt und...
66292395889820447de113ee    @Gelber Roboter, während ich Russlands Bemühun...
66291ead889820447de112d1    In #Russland wird ein hochrangiger Militärbeam...
66252bb4616a4cb7061ee4c2    Schade, dass diplomatische Chancen auf Frieden...
662623e283ad1b524033ba5a    Absolut großartig! Eine moderne Interpretation...
                                                  ...                        
66278754b8beabb5822948a9    Kiew gerät in der Ostukraine immer weiter unte...
66284fa3810b4c35f2bb3d73                                                 Post
6629f9f037d6395f42ca6d64                                                Ja ja
6629392037d6395f42ca21b5             Wir haben alles dabei - so richtig April
662678a9d4ec55a25b931049    @LilaBlume, guter Punkt. Wahre Solidarität erf...
Name: content, Length: 1841, dtype: object

In [127]:
dataset.to_csv(CFG.data_dir / "final" / "dataset_for_annotation.csv")

In [19]:
dataset = (
    pd.read_csv(
        CFG.data_dir / "final" / "dataset_annotated.csv", 
        index_col=["id"]
    )
    .join(
        pd.read_parquet(CFG.final_data_files["user_content"])
        .drop(columns=['content']),
        how="inner"
    )
    .dropna(subset=["feels_ai"])
)

In [22]:
dataset.groupby("userType")["feels_ai"].value_counts()

userType  feels_ai
bot       5.0         128
          4.0           9
          3.0           7
          1.0           1
human     1.0          33
          3.0           9
          2.0           7
          5.0           4
Name: count, dtype: int64

In [27]:
dataset[(dataset["userType"] == "human") & (dataset["feels_ai"] == 5.0)]["content"].tolist()

['Klickbait.',
 'Steinmeier in der Türkei',
 'Das Wetter wird endlich wieder besser',
 'Die deutsche Außenministerin steht unter Druck – sogar aus der eigenen Koalition 😨 https://www.bild.de/politik/ausland-und-internationales/riesen-kritik-an-baerbock-unglaublich-naiv-6624d45576790e1ca24f918d']